## Import Libraries and specify Amazon S3 Bucket

* `bucket` - An S3 bucket accessible by this account.
* `prefix` - The location in the bucket where this notebook's input and output data will be stored. (The default value is sufficient.)

In [138]:
import boto3
import botocore
import sagemaker
import sys

#Specify an S3 bucket that will be used for the training process.
bucket = 'sagemaker-walebadr'   # <--- specify a bucket you have access to
prefix = 'sagemaker/rcf-benchmarks'
execution_role = sagemaker.get_execution_role()


# check if the bucket exists
try:
    boto3.Session().client('s3').head_bucket(Bucket=bucket)
except botocore.exceptions.ParamValidationError as e:
    print('Hey! You either forgot to specify your S3 bucket'
          ' or you gave your bucket an invalid name!')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == '403':
        print("Hey! You don't have permission to access the bucket, {}.".format(bucket))
    elif e.response['Error']['Code'] == '404':
        print("Hey! Your bucket, {}, doesn't exist!".format(bucket))
    else:
        raise
else:
    print('Training input/output will be stored in: s3://{}/{}'.format(bucket, prefix))

Training input/output will be stored in: s3://sagemaker-walebadr/sagemaker/rcf-benchmarks


## Obtain and Inspect Example Data


Our data comes from the Numenta Anomaly Benchmark (NAB) [[1](https://github.com/numenta/NAB/blob/master/data/realKnownCause/ambient_temperature_system_failure.csv)]. The data records the temperature sensor data from an internal component of a large industrial machine. The data collected over the course of 3 months aggregated into 5-minute buckets.

> https://github.com/numenta/NAB/blob/master/data/realKnownCause/ambient_temperature_system_failure.csv

In [133]:
%%time

import pandas as pd
import urllib.request

data_filename = 'machine_temperature_system_failure.csv'
data_source = 'https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/machine_temperature_system_failure.csv'

urllib.request.urlretrieve(data_source, data_filename)
taxi_data = pd.read_csv(data_filename, delimiter=',')
prediction_data = pd.read_csv("temp_predictions.csv", delimiter=',')


CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 137 ms


## Data Inspection

Before training any models it is important to inspect our data, first. Perhaps there are some underlying patterns or structures that we could provide as "hints" to the model or maybe there is some noise that we could pre-process away. The raw data looks like this:

In [119]:
%%time

taxi_data.head()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 302 µs


,timestamp,value
0,2013-12-02 21:15:00,73.967322
1,2013-12-02 21:20:00,74.935882
2,2013-12-02 21:25:00,76.124162
3,2013-12-02 21:30:00,78.140707
4,2013-12-02 21:35:00,79.329836


Human beings are visual creatures so let's take a look at a plot of the data.

In [134]:
import bokeh
import bokeh.io
from bokeh.models import HoverTool
bokeh.io.output_notebook()
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
import bokeh.palettes
taxi_data['timestamp'] = pd.to_datetime(taxi_data['timestamp'])
output_file("datetime.html")
date = taxi_data['timestamp']
temp = taxi_data['value']

x = date.values
y = temp.values
score = []

string_x = list(map(str, x))
source = ColumnDataSource(taxi_data)
source.add(taxi_data['timestamp'].apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S')), 'event_date_formatted')
#Hover Tool

hover = HoverTool(
    names = ["temp"],
    tooltips=[
        ( 'date',   '@event_date_formatted'),
        ( 'temperature',  '$y' ),        
    ],


    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

p = figure(x_axis_type='datetime', plot_width=900, plot_height=400, tools=[hover, 'pan','wheel_zoom','box_zoom','reset']) 
p.line( name="temp", x='timestamp',y= 'value',source=source, line_width=2,color='navy', alpha=0.5)
show(p)

Loading BokehJS ...

## Store Data on S3

The Random Cut Forest Algorithm accepts data in [RecordIO](https://mxnet.apache.org/api/python/io/io.html#module-mxnet.recordio) [Protobuf](https://developers.google.com/protocol-buffers/) format. The SageMaker Python API provides helper functions for easily converting your data into this format. Below we convert the taxicab data and upload it to the `bucket + prefix` Amazon S3 destination specified at the beginning of this notebook in the [Setup AWS Credentials](#Setup-AWS-Credentials) section.

In [136]:
def convert_and_upload_training_data(ndarray, bucket, prefix, filename='data.pbr'):
    import boto3
    import os
    from sagemaker.amazon.common import numpy_to_record_serializer
    
    # convert numpy array to Protobuf RecordIO format
    serializer = numpy_to_record_serializer()
    buffer = serializer(ndarray)
    
    # Upload to S3
    s3_object = os.path.join(prefix, 'train', filename)
    boto3.Session().resource('s3').Bucket(bucket).Object(s3_object).upload_fileobj(buffer)
    
    s3_path = 's3://{}/{}'.format(bucket, s3_object)
    return s3_path

# RCV only works on an array of values.
s3_train_data = convert_and_upload_training_data(
    taxi_data.value.as_matrix().reshape(-1,1),
    bucket,
    prefix)
print('Uploaded data to {}'.format(s3_train_data))

Uploaded data to s3://sagemaker-walebadr/sagemaker/rcf-benchmarks/train/data.pbr


# Training

***

We have created a training data set and uploaded it to S3. Next, we configure a SageMaker training job to use the Random Cut Forest (RCF) algorithm on said training data.

The first step is to specify the location of the Docker image containing the SageMaker Random Cut Forest algorithm. In order to minimize communication latency, we provide containers for each AWS region in which SageMaker is available. The code below automatically chooses an algorithm container based on the current region; that is, the region in which this notebook is run.

In [117]:
import boto3

# select the algorithm container based on this notebook's current location
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/randomcutforest:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/randomcutforest:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/randomcutforest:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/randomcutforest:latest'}
region_name = boto3.Session().region_name
container = containers[region_name]

print('Using SageMaker RCF container: {} ({})'.format(container, region_name))

Using SageMaker RCF container: 382416733822.dkr.ecr.us-east-1.amazonaws.com/randomcutforest:latest (us-east-1)


## Hyperparameters

Particular to a SageMaker RCF training job are the following hyperparameters:

* **`num_samples_per_tree`** - the number randomly sampled data points sent to each tree. As a general rule, `1/num_samples_per_tree` should approximate the the estimated ratio of anomalies to normal points in the dataset.
* **`num_trees`** - the number of trees to create in the forest. Each tree learns a separate model from different samples of data. The full forest model uses the mean predicted anomaly score from each constituent tree.
* **`feature_dim`** - the dimension of each data point.

In addition to these RCF model hyperparameters, we provide additional parameters defining things like the EC2 instance type on which training will run, the S3 bucket containing the data, and the AWS access role. Note that,

* Recommended instance type: `ml.m4`, `ml.c4`, or `ml.c5`
* Current limitations:
  * The RCF algorithm does not take advantage of GPU hardware.

In [121]:
%%time

session = sagemaker.Session()
input_mode = "Pipe"
# specify general training job information
rcf = sagemaker.estimator.Estimator(
    container,
    execution_role,
    input_mode=input_mode,
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    sagemaker_session=session,
)

# set algorithm-specific hyperparameters
rcf.set_hyperparameters(
    num_samples_per_tree = 200,
    num_trees = 50,
    feature_dim = 1,
)

# RCF training requires sharded data. See documentation for
# more information.
s3_train_input = sagemaker.session.s3_input(
    s3_train_data,
    distribution='ShardedByS3Key',
    content_type='application/x-recordio-protobuf',
)

# run the training job on input data stored in S3

rcf.fit({'train': s3_train_input})

INFO:sagemaker:Creating training-job with name: randomcutforest-2018-05-27-02-56-28-445


..........................................
Docker entrypoint called with argument(s): train
[05/27/2018 02:59:52 INFO 140498708735808] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'num_samples_per_tree': 256, u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'_kvstore': u'dist_async', u'force_dense': u'true', u'epochs': 1, u'num_trees': 100, u'eval_metrics': [u'accuracy', u'precision_recall_fscore'], u'_num_kv_servers': u'auto', u'mini_batch_size': 1000}
[05/27/2018 02:59:52 INFO 140498708735808] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'1', u'num_samples_per_tree': u'200', u'num_trees': u'50'}
[05/27/2018 02:59:52 INFO 140498708735808] Final configuration: {u'num_samples_per_tree': u'200', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'_kvstore': u'dist_async', u'force_dense': u'true', u'epoch

===== Job Complete =====
Billable seconds: 121
CPU times: user 236 ms, sys: 32 ms, total: 268 ms
Wall time: 4min 7s


If you see the message

> `===== Job Complete =====`

at the bottom of the output logs then that means training successfully completed and the output RCF model was stored in the specified output path. You can also view information about and the status of a training job using the AWS SageMaker console. Just click on the "Jobs" tab and select training job matching the training job name, below:

In [137]:
print('Training job name: {}'.format(rcf.latest_training_job.job_name))

Training job name: randomcutforest-2018-05-27-02-56-28-445


# Inference

***

A trained Random Cut Forest model does nothing on its own. We now want to use the model we computed to perform inference on data. In this case, it means computing anomaly scores from input time series data points.

We create an inference endpoint using the SageMaker Python SDK `deploy()` function from the job we defined above. We specify the instance type where inference is computed as well as an initial number of instances to spin up. We recommend using the `ml.c5` instance type as it provides the fastest inference time at the lowest cost.

In [122]:
rcf_inference = rcf.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
)

INFO:sagemaker:Creating model with name: randomcutforest-2018-05-27-03-11-09-463
INFO:sagemaker:Creating endpoint with name randomcutforest-2018-05-27-02-56-28-445


--------------------------------------------------------------!

Congratulations! You now have a functioning SageMaker RCF inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console and selecting the endpoint matching the endpoint name, below: 

In [123]:
print('Endpoint name: {}'.format(rcf_inference.endpoint))

Endpoint name: randomcutforest-2018-05-27-02-56-28-445


## Data Serialization/Deserialization

We can pass data in a variety of formats to our inference endpoint. In this example we will demonstrate passing CSV-formatted data. Other available formats are JSON-formatted and RecordIO Protobuf. We make use of the SageMaker Python SDK utilities `csv_serializer` and `json_deserializer` when configuring the inference endpoint.

In [124]:
from sagemaker.predictor import csv_serializer, json_deserializer

rcf_inference.content_type = 'text/csv'
rcf_inference.serializer = csv_serializer
rcf_inference.deserializer = json_deserializer

Let's pass the training dataset, in CSV format, to the inference endpoint so we can automatically detect the anomalies we saw with our eyes in the plots, above. Note that the serializer and deserializer will automatically take care of the datatype conversion from Numpy NDArrays.

For starters, let's only pass in the first six datapoints so we can see what the output looks like.

In [126]:
import time
prediction_data = pd.read_csv("machine_temperature_system_failure.csv", delimiter=',')
prediction_data_numpy = prediction_data.value.as_matrix().reshape(-1,1)
results = rcf_inference.predict(prediction_data_numpy)

## Computing Anomaly Scores

Now, let's compute and plot the anomaly scores from the entire taxi dataset.

In [127]:
results = rcf_inference.predict(prediction_data_numpy)
scores = [datum['score'] for datum in results['scores']]

# add scores to taxi data frame and print first few values
prediction_data['score'] = pd.Series(scores, index=prediction_data.index)
anomalies = prediction_data[prediction_data['score'] > score_cutoff]
sorted_anomalies = anomalies.sort_values(by=['score'], ascending=False)

sorted_anomalies[:10]

,timestamp,value,score
3986,2013-12-16 17:25:00,2.084721,4.374799
3985,2013-12-16 17:20:00,4.117241,4.211207
3984,2013-12-16 17:15:00,6.440238,4.104767
3983,2013-12-16 17:10:00,6.918645,4.081857
3982,2013-12-16 17:05:00,10.001966,3.951057
3981,2013-12-16 17:00:00,9.633952,3.944658
3987,2013-12-16 17:30:00,12.120381,3.814362
3980,2013-12-16 16:55:00,12.414093,3.798221
3979,2013-12-16 16:50:00,14.671060,3.667530
3978,2013-12-16 16:45:00,16.457109,3.554547


In [128]:
from bokeh.models import Range1d, LinearAxis
def prediction(data):
    prediction_data = data
    prediction_data['timestamp'] = pd.to_datetime(prediction_data['timestamp'])
    output_file("datetime.html")
    temp = prediction_data['value']
    scores = prediction_data['score']

    hover = HoverTool(
        names = ["temp","score"],
        tooltips=[
            ( 'date',   '@event_date_formatted'),
            ( 'temp',  '@temp_y' ), 
           ( 'Score',  '@score_y' ), 
        ],
    )
    source = ColumnDataSource(prediction_data)
    source.add(prediction_data['timestamp'].apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S')), 'event_date_formatted')
    source.add(temp, 'temp_y')
    source.add(scores, 'score_y')

    p = figure(x_axis_type='datetime', plot_width=1000, plot_height=350, tools=[hover, 'pan','wheel_zoom','box_zoom','reset']) 
    p.line( name = "temp", x='timestamp',y= 'value',source=source, line_width=2,color='navy', alpha=0.5, legend=["Temperature"])
    p.extra_y_ranges = {"Anomaly": Range1d(start=0, end=10)}
    p.add_layout(LinearAxis(y_range_name="Anomaly"), 'right')
    p.line( name = "score", x='timestamp',y='score',source=source, line_width=2,color='red', alpha=0.5, y_range_name="Anomaly",legend=["Score"])
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    p.title.text = "Anomaly Detection for Device Temperature"



    #select the highest anomaly scores
    score_mean = prediction_data['score'].mean()
    score_std = prediction_data['score'].std()
    score_cutoff = score_mean + 3*score_std

    anomalies = prediction_data[prediction_data['score'] > score_cutoff]
    sorted_anomalies = anomalies.sort_values(by=['score'], ascending=False)

    source = ColumnDataSource(prediction_data)
    #p.circle( sorted_anomalies['timestamp'],sorted_anomalies['score'], line_width=8,color='black', alpha=0.5, y_range_name="Anomaly")
    p.circle( sorted_anomalies['timestamp'],sorted_anomalies['value'], line_width=5,color='black')


    show(p)
    
def process_data(file):
    prediction_data = pd.read_csv(file, delimiter=',')
    prediction_data_numpy = prediction_data.value.as_matrix().reshape(-1,1)
    results = rcf_inference.predict(prediction_data_numpy)
    scores = [datum['score'] for datum in results['scores']]
    prediction_data['score'] = pd.Series(scores, index=prediction_data.index)
    return prediction_data
#prediction(prediction_data)

## Make prediction on the whole dataset

In [129]:
prediction(process_data("machine_temperature_system_failure.csv"))

## Make prediction on a new data

In [130]:
prediction(process_data("temp_predictions.csv"))

Note that the anomaly score spikes where our eyeball-norm method suggests there is an anomalous data point as well as in some places where our eyeballs are not as accurate.

Below we print and plot any data points with scores greater than 3 standard deviations (approx 99.9th percentile) from the mean score.

The first anomaly was a planned shutdown. The third anomaly was a catastrophic system failure. The other few anomalies in the middle, a subtle but observable change in the behavior, indicated the actual onset of the problem that led to the eventual system failure.

* `2013-12-16` - A planned shutdown
* `2014-02-01` - Subtle anomalies that led to the failure.
* `2014-02-08` - Catastrophic System Failure

Note that our algorithm managed to capture these events along with quite a few others. Below we add these anomalies to the score plot.

With the current hyperparameter choices we see that the three-standard-deviation threshold, while able to capture the known anomalies as well as the ones apparent in the ridership plot, is rather sensitive to fine-grained peruturbations and anomalous behavior. Adding trees to the SageMaker RCF model could smooth out the results as well as using a larger data set.

## Stop and Delete the Endpoint

Finally, we should delete the endpoint before we close the notebook.

To do so execute the cell below. Alternately, you can navigate to the "Endpoints" tab in the SageMaker console, select the endpoint with the name stored in the variable `endpoint_name`, and select "Delete" from the "Actions" dropdown menu. 

In [63]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)

INFO:sagemaker:Deleting endpoint with name: randomcutforest-2018-05-20-23-26-13-719


# Epilogue

---

We used Amazon SageMaker Random Cut Forest to detect anomalous datapoints in a taxi ridership dataset. In these data the anomalies occurred when ridership was uncharacteristically high or low. However, the RCF algorithm is also capable of detecting when, for example, data breaks periodicity or uncharacteristically changes global behavior.

Depending on the kind of data you have there are several ways to improve algorithm performance. One method, for example, is to use an appropriate training set. If you know that a particular set of data is characteristic of "normal" behavior then training on said set of data will more accurately characterize "abnormal" data.

Another improvement is make use of a windowing technique called "shingling". This is especially useful when working with periodic data with known period, such as the NYC taxi dataset used above. The idea is to treat a period of $P$ datapoints as a single datapoint of feature length $P$ and then run the RCF algorithm on these feature vectors. That is, if our original data consists of points $x_1, x_2, \ldots, x_N \in \mathbb{R}$ then we perform the transformation,

```
data = [[x_1],            shingled_data = [[x_1, x_2, ..., x_{P}],
        [x_2],    --->                     [x_2, x_3, ..., x_{P+1}],
        ...                                ...
        [x_N]]                             [x_{N-P}, ..., x_{N}]]

```

In [ ]:
prediction_data = pd.read_csv("machine_temperature_system_failure.csv", delimiter=',')
shingle(prediction_data.values[:,1], 20)

In [65]:
import numpy as np

def shingle(data, shingle_size):
    num_data = len(data)
    shingled_data = np.zeros((num_data-shingle_size, shingle_size))
    
    for n in range(num_data - shingle_size):
        shingled_data[n] = data[n:(n+shingle_size)]
    return shingled_data

# single data with shingle size=48 (one day) and upload to S3
shingle_size = 48
prefix_shingled = 'sagemaker/randomcutforest_shingled'
prediction_data_shingled = shingle(prediction_data.values[:,1], shingle_size)
s3_train_data_shingled = convert_and_upload_training_data(
    prediction_data_shingled,
    bucket,
    prefix_shingled)

print(prediction_data_shingled)
print(s3_train_data_shingled)

[[73.96732207 74.935882   76.12416182 ... 83.5464428  83.48998999
  83.43534304]
 [74.935882   76.12416182 78.14070732 ... 83.48998999 83.43534304
  85.18336642]
 [76.12416182 78.14070732 79.32983574 ... 83.43534304 85.18336642
  85.34462435]
 ...
 [91.00585235 90.72704516 90.02528249 ... 97.36090483 98.18541493
  97.80416849]
 [90.72704516 90.02528249 91.55674386 ... 98.18541493 97.80416849
  97.13546835]
 [90.02528249 91.55674386 91.80333835 ... 97.80416849 97.13546835
  98.05685212]]
s3://sagemaker-walebadr/sagemaker/randomcutforest_shingled/train/data.pbr


We create a new training job and and inference endpoint. (Note that we cannot re-use the endpoint created above because it was trained with one-dimensional data.)

In [66]:
session = sagemaker.Session()

rcf = sagemaker.estimator.Estimator(
    container,
    execution_role,
    output_path='s3://{}/{}/output'.format(bucket, prefix_shingled),
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    sagemaker_session=session,
)

rcf.set_hyperparameters(
    num_samples_per_tree = 200,
    num_trees = 50,
    feature_dim = shingle_size,
)

s3_train_input = sagemaker.session.s3_input(
    s3_train_data_shingled,
    distribution='ShardedByS3Key',
    content_type='application/x-recordio-protobuf',
)

rcf.fit({'train': s3_train_input})

INFO:sagemaker:Creating training-job with name: randomcutforest-2018-05-21-00-03-40-241


......................................
Docker entrypoint called with argument(s): train
[05/21/2018 00:06:43 INFO 140318202361664] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'num_samples_per_tree': 256, u'_num_gpus': u'auto', u'_log_level': u'info', u'_kvstore': u'dist_async', u'force_dense': u'true', u'epochs': 1, u'num_trees': 100, u'eval_metrics': [u'accuracy', u'precision_recall_fscore'], u'_num_kv_servers': u'auto', u'mini_batch_size': 1000}
[05/21/2018 00:06:43 INFO 140318202361664] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'48', u'num_samples_per_tree': u'200', u'num_trees': u'50'}
[05/21/2018 00:06:43 INFO 140318202361664] Final configuration: {u'num_samples_per_tree': u'200', u'_num_gpus': u'auto', u'_log_level': u'info', u'_kvstore': u'dist_async', u'force_dense': u'true', u'epochs': 1, u'feature_dim': u'48', u'num_trees': u'50', u'eval_metrics': [u'

===== Job Complete =====
Billable seconds: 103


In [67]:
from sagemaker.predictor import csv_serializer, json_deserializer

rcf_inference = rcf.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
)

rcf_inference.content_type = 'text/csv'
rcf_inference.serializer = csv_serializer
rcf_inference.deserializer = json_deserializer

INFO:sagemaker:Creating model with name: randomcutforest-2018-05-21-00-07-17-043
INFO:sagemaker:Creating endpoint with name randomcutforest-2018-05-21-00-03-40-241


---------------------------------------------------------------!

Using the above inference endpoint we compute the anomaly scores associated with the shingled data.

In [86]:
rcf_inference.content_type = 'text/csv'
rcf_inference.serializer = csv_serializer
rcf_inference.deserializer = json_deserializer

prediction_data_shingled = pd.read_csv("temp_predictions.csv", delimiter=',')
#prediction_data_shingled = shingle(prediction_data.values[:,1], 1)
pred_data = prediction_data.value.as_matrix().reshape(-1,48)
results = rcf_inference.predict(pred_data)
print(pred_data)

[[ 66.89615854  66.55123626  67.97364915  66.11182479  67.39853634
   65.59692094  65.31088565  66.94188976  65.96888298  65.12423914
   66.11789457  64.80124145  66.39476737  65.05548849  65.42792911
   65.79025781  65.24156143  64.77355145  64.25844658  63.32511037
   63.92825593  62.71478623  63.5697321   62.01093406  62.35603729
   63.29512278  61.51612039  61.50167623  61.49128902  63.12723689
   61.41971741  62.49812045  62.62345077  62.01165423  61.11837772
   59.77550561  61.50966304  59.66600129  59.57919588  59.12470295
   60.24540836  60.25964641  58.81060689  59.64776118  58.86810738
   59.59463032  58.60295114  58.67059847]
 [ 57.61151954  58.80721121  57.69286607  56.74915104  57.91419321
   57.56596634  56.72141269  57.84285288  57.02785876  56.93371332
   57.44456351  57.43423073  55.01842643  55.73219618  54.61028367
   55.08287152  55.56345357  54.42607397  55.77414637  54.7775747
   55.45028914  53.85600716  53.56473816  53.68843595  53.29903854
   54.69347371  54.62

In [90]:
#print(prediction_data_shingled[:3])
#results = rcf_inference.predict(prediction_data_shingled)
#scores = np.array([datum['score'] for datum in results['scores']])
#prediction_data_shingled_data = prediction_data
scores = [datum['score'] for datum in results['scores']]
print(scores)
print(prediction_data_shingled)
#prediction_data_shingled['score'] = pd.Series(scores, index=prediction_data_shingled.index)
print(prediction_data_shingled)
# compute the shingled score distribution and cutoff and determine anomalous scores
score_mean = prediction_data_shingled.score.mean()
score_std = prediction_data_shingled.score.std()
score_cutoff = score_mean + score_std

anomalies = scores[scores > score_cutoff]
#anomaly_indices = np.arange(len(scores))[scores > score_cutoff]
#sorted_anomalies = anomalies.sort_values(by=['score'], ascending=False)


[0.9242281394, 1.0865454079, 1.4281386138, 1.6779477229, 2.6735863967, 0.7867350102]
               timestamp       value
0    2013-12-16 00:00:00   66.896159
1    2013-12-16 00:05:00   66.551236
2    2013-12-16 00:10:00   67.973649
3    2013-12-16 00:15:00   66.111825
4    2013-12-16 00:20:00   67.398536
5    2013-12-16 00:25:00   65.596921
6    2013-12-16 00:30:00   65.310886
7    2013-12-16 00:35:00   66.941890
8    2013-12-16 00:40:00   65.968883
9    2013-12-16 00:45:00   65.124239
10   2013-12-16 00:50:00   66.117895
11   2013-12-16 00:55:00   64.801241
12   2013-12-16 01:00:00   66.394767
13   2013-12-16 01:05:00   65.055488
14   2013-12-16 01:10:00   65.427929
15   2013-12-16 01:15:00   65.790258
16   2013-12-16 01:20:00   65.241561
17   2013-12-16 01:25:00   64.773551
18   2013-12-16 01:30:00   64.258447
19   2013-12-16 01:35:00   63.325110
20   2013-12-16 01:40:00   63.928256
21   2013-12-16 01:45:00   62.714786
22   2013-12-16 01:50:00   63.569732
23   2013-12-16 01:55:00   

AttributeError: 'DataFrame' object has no attribute 'score'

Finally, we plot the scores from the shingled data on top of the original dataset and mark the score lying above the anomaly score threshold.

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

#
# *Try this out* - change `start` and `end` to zoom in on the 
# anomaly found earlier in this notebook
#
start, end = 0, len(taxi_data)
taxi_data_subset = taxi_data[start:end]

ax1.plot(taxi_data['value'], color='C0', alpha=0.8)
ax2.plot(scores, color='C1')
ax2.scatter(anomaly_indices, anomalies, color='k')

ax1.grid(which='major', axis='both')
ax1.set_ylabel('Taxi Ridership', color='C0')
ax2.set_ylabel('Anomaly Score', color='C1')
ax1.tick_params('y', colors='C0')
ax2.tick_params('y', colors='C1')
ax1.set_ylim(0, 100)
ax2.set_ylim(min(scores), 1.4*max(scores))
fig.set_figwidth(10)

We see that with this particular shingle size, hyperparameter selection, and anomaly cutoff threshold that the shingled approach more clearly captures the major anomalous events: the spike at around t=6000 and the dip at around t=10000. In general, the number of trees, sample size, and anomaly score cutoff are all parameters that a data scientist may need experiment with in order to achieve desired results. The use of a labeled test dataset allows the used to obtain common accuracy metrics for anomaly detection algorithms. For more information about Amazon SageMaker Random Cut Forest see the [AWS Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html).

In [ ]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)